In [14]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats 

In [12]:
x_1 = -9.3 
sigmax_1 = 8.2
y_1 = -1.3
sigmay_1 = 8.4
x_2 = 5.7 
sigmax_2 = 8.2
y_2 = 6.5
sigmay_2 = 8.3
rhox1y1 = -0.1
rhox1y2 = 0.1
rhox1x2 = -0.05
rhox2y1 = 0.1
rhox2y2 = 0.1
rhoy1y2 = -0.05


In [13]:
cov_matrix = [[sigmax_1**2, rhox1y1*sigmax_1*sigmay_1, rhox1x2*sigmax_1*sigmax_2, rhox1y2*sigmax_1*sigmay_2],
              [ rhox1y1*sigmax_1*sigmay_1,sigmay_1**2, rhox2y1*sigmay_1*sigmax_2, rhoy1y2*sigmay_1*sigmay_2 ],
              [ rhox1x2*sigmax_1*sigmax_2, rhox2y1*sigmay_1*sigmax_2, sigmax_2**2, rhox2y2*sigmax_2*sigmay_2 ],
              [rhox1y2*sigmax_1*sigmay_2, rhoy1y2*sigmay_1*sigmay_2, rhox2y2*sigmax_2*sigmay_2, sigmay_2**2]]

In [16]:
f4 = scipy.stats.multivariate_normal([x_1,y_1,x_2,y_2], cov_matrix) 

In [17]:
def XY(r,delta,gamma):
    X_1 = r*np.cos(delta+gamma)
    Y_1 = r*np.sin(delta+gamma)
    X_2 = r*np.cos(delta-gamma)
    Y_2 = r*np.sin(delta-gamma)
    return X_1,Y_1,X_2,Y_2